In [1]:
# imports, loading the neccessary libraries

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Tell me a joke about a programmer"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Why do programmers prefer dark mode?

Because light attracts bugs!


In [5]:
# Define a class to represent a website
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using Selenium and BeautifulSoup
        """
        self.url = url
        
        # Set up Chrome options
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run in background
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
        
        # Initialize the driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        try:
            # Navigate to the URL
            driver.get(url)
            
            # Wait for the page to load (adjust time as needed)
            time.sleep(5)
            
            # Get the page source after JavaScript execution
            html_content = driver.page_source
            
            # Parse with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            
            self.title = soup.title.string if soup.title else "No title found"
            
            # Remove irrelevant elements
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            
            self.text = soup.body.get_text(separator="\n", strip=True)
            
        finally:
            # Always close the driver
            driver.quit()

# Test it out
ed = Website("https://www.ifrs17game.com/")
print(ed.title)
print(ed.text[:500])  # Print first 500 characters

IFRS 17 Quest and Conquer
You need to enable JavaScript to run this app.
G
Guest User
Evaluating IFRS 17
Sign In
Sign Up
IFRS 17 Quest and Concur: Regulatory Training Game
Score
0
Streak
0
Level 1
Combo
x1
View Leaderboard
IFRS 17 Training Modules
(0/10 completed)
📚
IFRS 17 Fundamentals
In Progress
🎯
Combination & Separation of Insurance Contracts
Sign up to unlock
📊
Level of Aggregation
Sign up to unlock
📏
Recognition of Insurance Contracts
Sign up to unlock
🔒
Measurement on Initial Recognition
Sign up to unlock
🚀
Subse


In [6]:
# Define a system prompt for the assistant
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled IFRS 17 Quest and Conquer
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

You need to enable JavaScript to run this app.
G
Guest User
Evaluating IFRS 17
Sign In
Sign Up
IFRS 17 Quest and Concur: Regulatory Training Game
Score
0
Streak
0
Level 1
Combo
x1
View Leaderboard
IFRS 17 Training Modules
(0/10 completed)
📚
IFRS 17 Fundamentals
In Progress
🎯
Combination & Separation of Insurance Contracts
Sign up to unlock
📊
Level of Aggregation
Sign up to unlock
📏
Recognition of Insurance Contracts
Sign up to unlock
🔒
Measurement on Initial Recognition
Sign up to unlock
🚀
Subsequent Measurement
Sign up to unlock
🔄
Discounting CSM and Risk Adjustment
Sign up to unlock
⚠️
Onerous Contracts
Sign up to unlock
📋
Premium Allocation Approach
Sign up to unlock
🔀
Reinsurance Contracts Held
Sign up to unlock
IFRS 17 Fundamentals - Question 1/10
0:00
(

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [9]:
messages = [
    {"role": "system", "content": "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."},
    {"role": "user", "content": "You are looking at a website titled {website.title}. The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too."}
]

In [10]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Please provide the content of the website, and I will summarize it for you.


In [11]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled IFRS 17 Quest and Conquer\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nYou need to enable JavaScript to run this app.\nG\nGuest User\nEvaluating IFRS 17\nSign In\nSign Up\nIFRS 17 Quest and Concur: Regulatory Training Game\nScore\n0\nStreak\n0\nLevel 1\nCombo\nx1\nView Leaderboard\nIFRS 17 Training Modules\n(0/10 completed)\n📚\nIFRS 17 Fundamentals\nIn Progress\n🎯\nCombination & Separation of Insurance Contracts\nSign up to unlock\n📊\nLevel of Aggregation\nSign up to unlock\n📏\nRecognition of Insurance Contracts\nSign up to unlock\n🔒\nMeasurement on Initial Recognition\nSign up to unlock\n🚀\nSubsequen

In [13]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://www.ifrs17game.com/")

'# Summary of IFRS 17 Quest and Conquer\n\nThe website **IFRS 17 Quest and Conquer** is an interactive training platform focused on educating users about IFRS 17, the international financial reporting standard for insurance contracts. The site features a training game designed to help users learn the fundamentals and complexities of IFRS 17 through various levels and modules.\n\n### Key Features\n- **Training Modules**: The platform includes modules on topics like the combination and separation of insurance contracts, level of aggregation, recognition and measurement of insurance contracts, and more. However, most modules require the user to sign up to access.\n- **Interactive Learning**: Users can track their progress with scores, streaks, and completions of modules. Currently, users can see their performance in terms of completed training modules.\n- **Questionnaire**: There’s a quiz section to test knowledge on IFRS 17 basics, like what IFRS 17 replaces.\n\n### Notes\n- The site is 

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://www.ifrs17game.com/")

# Summary of IFRS 17 Quest and Conquer

**Website Overview:**
The "IFRS 17 Quest and Conquer" website serves as an interactive training platform focused on the IFRS 17 standards related to insurance contracts. It offers a game-like experience for users to learn key concepts and test their knowledge through various training modules.

**Training Modules Available:**
- **IFRS 17 Fundamentals** - Currently in progress.
- Other modules include topics such as:
  - Combination & Separation of Insurance Contracts (locked)
  - Level of Aggregation (locked)
  - Recognition of Insurance Contracts (locked)
  - Measurement on Initial Recognition (locked)
  - Subsequent Measurement (locked)
  - Discounting CSM and Risk Adjustment (locked)
  - Onerous Contracts (locked)
  - Premium Allocation Approach (locked)
  - Reinsurance Contracts Held (locked)

**User Engagement:**
The platform tracks user progress, including scores, streaks, and levels of completion for the training modules.

**Content Interaction:**
Users can participate in quizzes (e.g., a question about contract exclusions from IFRS 17) that enhance learning and test understanding in a gamified way.

**Additional Information:**
The platform is powered by Kenbright AI, indicating a modern and AI-driven approach to regulatory training.